In [1]:
import json
import csv
import tweepy
import re

In [6]:
# load Twitter API credentials

with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

In [11]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [29]:
def search_for_hashtags(hashtag_phrase):
    
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s.csv' % (fname), 'a',encoding='utf-8') as file:

        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags'])

        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                   lang="en", tweet_mode='extended',encodint='utf-8').items(1000):
            w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8').decode('utf-8'), tweet.user.screen_name.encode('utf-8').decode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']]])

In [13]:
hashtag_phrase = '#FacebookDown #instagramdown'

In [30]:
search_for_hashtags(hashtag_phrase)

In [31]:
import time
while True:
    try:
        search_for_hashtags(hashtag_phrase)
        time.sleep(10)
    except TweepError:
        time.sleep(60*15)

TweepError: Twitter error response: status code = 429